# 相关库导入

In [ ]:
import time
from charm.toolbox.integergroup import RSAGroup
from charm.schemes.pkenc.pkenc.paillier99 import Ciphertext, Pai99
import keras
import numpy as np
from keras.datasets import mnist
from sklearn .svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

# 数据集处理

In [ ]:
(x_train,y_train),(x_test,y_test) = mnist.load_data() #将训练集和测试集合并
x = np.concatenate((x_train,x_test),axis = 0)
y = np.concatenate((y_train,y_test),axis = 0)
x=x.reshape(70000,784)
x= x.astype (' float32')
x /= 255

In [ ]:
x_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.1,random_state = 666)
iterations = 10
clients = 11

# 生成公钥和私钥

In [ ]:
file = open("consuming times.txt","a")
start = time.time ()
group=RSAGroup()
pai=Pai99(group)
pk,sk = pai.keygen(secparam=128)
gen_public_param_time = time.time()-start

# 针对每个客户端获得训练出的模型

In [ ]:
for i in range(iterations):
# 每个 client 需要用各自的第  份数据训练模型
    start = time.time()
    for j in range(clients):
        models[j].partial_fit(train_x[j][i],train_y[j][i],classes = np.array([0,1,2,3,4,5,6,7,8,9]))
    gen_model_time = time.time()-start
    local_train_time += gen_model_time
    print(i," the iteration fit Ok")
    y_predict = models[0].predict (x_test)
    print(i," the accuracy:",accuracy_score(y_test,y_predict))
    acc_list.append(accuracy_score(y_test,y_predict))
    # 保存每个模型的参数
    model_weights =[]
    # 开始训练模型
    for j in range(clients):
        model_weights .append (get_1D_modelWeights_SVM(models[j]))

# 对每个模型进行加密

In [ ]:
start = time .time ()
for j in range(clients):
    print(j)
    print("svm:client"+":enc")
    enc_model = Encrypt_model(pai,pk,model_weights[j])
    print("enc_model len:",len(enc_model))
    enc_models.append(enc_model)

# 聚合计算

In [ ]:
all_sum_model = []
for k in range(len(enc_models[0])):
    temp_sum=pai.encrypti(pk,0)
    for j in range(clients):
            temp_sum+= enc_models[j][k]
    all_sum_model.append(temp_sum)

# 模型解密。将聚合后的参数传回客户端进行解密

In [ ]:
for j in range(clients):
    print(str(j)+"client")
    temp = Decrypt_list(pai,all_sum_model,sk,pk)
    temp = TransformModel_into_float(temp)
    temp = np.array(temp)
    temp = temp/clients
    coefs,intercepts = recover_svm_from_1D(models[j],temp)
    models[j].coef_= coefs
    models[j].intercept_= intercepts